In [41]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression

As explained in the report, hardcode latencies for critical points and establish a function to obtain the CPU latency between Core #0 and Core#p. Such function trivially interpolates the CPU latency assigning the same value found at the critical point until the next one is reached. 

In [42]:
# Latency values for different configurations
latencies = {
    "1": 0.10,  
    "4": 0.20,   
    "8": 0.30,   
    "16": 0.50,   
    "24": 0.80,  
    "48": 2.00   
}

# Function to get latency based on number of processes
def get_latency(p):
    for k in latencies.keys():
        if int(k) >= p:
            return latencies[k]
    return latencies[list(latencies.keys())[-1]]

# Create DataFrame for latency data
df = pd.DataFrame(data={
    "cpu": range(2, 48),
    "avg_latency": [get_latency(i) for i in range(2, 48)]
}).dropna()

Derive the time spent for a chain of np processes by converting the 0 -> p latencies into p_i -> p_(i + 1) latencies. As explained in the report, this is needed due to the implementation of the chain algorithm, based on the exchanges between neighbouring processes.  

In [43]:
# Function to derive the time spent for a chain of np processes
def get_chain_time(np):
    def get_latency(a, b):
        if a // 4 == b // 4:
            return latencies["1"]
        if a // 8 == b // 8:
            return latencies["4"]
        if a // 16 == b // 16:
            return latencies["8"]
        if a // 24 == b // 24:
            return latencies["16"]
        return latencies["24"]
    return sum([get_latency(i, i + 1) for i in range(np - 1)])

Load measured values for the Broadcast collective using the Chain algorithm and calculate its estimate based on an empirical formula, as explained in the report.

In [44]:
# Load actual latency data from file
actual = pd.read_csv("20240718_182957_thin_gather2_msize.txt", header=None, names=["actual_bcast"])
# Add actual latency data to DataFrame
df["actual_bcast"] = actual
# Add estimated broadcast latency to DataFrame
df["estimate_bcast"] = df.apply(lambda e: 0.6 * get_chain_time(int(e["cpu"]) - 1), axis=1)

In [45]:
# Fit a linear regression model to the actual latency data
# X = df.index.values.reshape(-1, 1)
# y = df["actual_bcast"]
# reg_model = LinearRegression()
# reg_model.fit(X, y)

In [46]:
df

,cpu,avg_latency,actual_bcast,estimate_bcast
0,2,0.2,0.20,0.00
1,3,0.2,0.40,0.06
2,4,0.2,0.22,0.12
3,5,0.3,0.27,0.18
4,6,0.3,0.38,0.30
5,7,0.3,0.45,0.36
6,8,0.3,0.57,0.42
7,9,0.5,1.02,0.48
8,10,0.5,0.92,0.66
9,11,0.5,1.03,0.72


In [47]:
# Plot actual vs estimated broadcast latency
fig = px.scatter(df, x="cpu", y="actual_bcast", title="Actual vs Estimated Broadcast Latency (Binary Tree Algorithm)", labels={"cpu": "Number of CPUs", "actual_bcast": "Latency (ms)"})
fig.add_trace(go.Scatter(x=df["cpu"], y=df["estimate_bcast"], mode="lines", name="Estimated Latency"))

# Add regression line to the plot
# reg_line = reg_model.predict(X)
# fig.add_trace(go.Scatter(x=df["cpu"], y=reg_line, mode="lines", name="Regression Line", line=dict(dash="dash")))

fig.show()